This is a script for building models for post-processing NWP visibility forecasts to produce calibrated deterministic forecasts using random forest regression (lead-time-separated approach). Please refer to Section 3.2.2 of the report for information of the lead-time-separated approach.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import pickle
import time

/Users/clifflau/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [1]:
# Change paths for own use
data_path = "ROOT/Data Preprocessings/"
model_export_path = "ROOT/trained_models/lead_time_separated/"
result_export_path = "ROOT/test_results_df/lead_time_separated/"

In [3]:
# List of input features (using ensemble means and std for visibility foreasts)
features = ['lead_hr', 'station_alt', 'dis_from_coast', 'time_in_day', 'day_in_year_sin', 'day_in_year_cos',
                    't_ensemble_mean', 't_ensemble_std',
                    'q_ensemble_mean', 'q_ensemble_std',
                    'tp6_ensemble_mean', 'tp6_ensemble_std',
                    'tcc_ensemble_mean', 'tcc_ensemble_std',
                    'wind_ensemble_mean', 'wind_ensemble_std',
                    'vis_ensemble_mean', 'vis_ensemble_std']

In [8]:
df_train = pd.read_csv(data_path + "df_training.csv", dtype={'time_in_day':'category'})
df_valid = pd.read_csv(data_path + "df_valid.csv", dtype={'time_in_day':'category'})
df_test = pd.read_csv(data_path + "df_test.csv", dtype={'time_in_day':'category'})
df_baseline = pd.read_csv(data_path + "df_training_baseline.csv", dtype={'time_in_day':'category'})

In [9]:
# Sample 10% of training records for faster training (comment out if necessary)
df_train = df_train.sample(frac=0.1, random_state=1)

In [15]:
for lead in np.arange(6,121,6):
    
    df_train_lead = df_train[df_train['lead_hr']==lead]
    df_valid_lead = df_valid[df_valid['lead_hr']==lead]
    df_test_lead = df_test[df_test['lead_hr']==lead]
    
    x_train = df_train_lead[features]
    y_train = df_train_lead['vis_obs']
    #y_train = np.log(df_train['vis_obs']+0.00001) # Log transformation (uncomment for transformation of output)

    x_valid = df_valid_lead[features]
    y_valid = df_valid_lead['vis_obs']
    #y_valid = df_valid['vis_obs']+0.00001 # Log transformation (uncomment for transformation of output)

    x_test = df_test_lead[features]
    y_test = df_test_lead['vis_obs']
    #y_valid = df_valid['vis_obs']+0.00001 # Log transformation (uncomment for transformation of output)

    # One-hot encode categorical 'time_in_day'
    x_train = x_train.join(pd.get_dummies(x_train['time_in_day']))
    x_train = x_train.drop('time_in_day',axis=1)
    x_valid = x_valid.join(pd.get_dummies(x_valid['time_in_day']))
    x_valid = x_valid.drop('time_in_day',axis=1)
    x_test = x_test.join(pd.get_dummies(x_test['time_in_day']))
    x_test = x_test.drop('time_in_day',axis=1)
    
    st = time.time()

    rf = RandomForestRegressor(n_estimators=100, max_depth=20, random_state=18)
    rf.fit(x_train, y_train)
    
    # Save the models
    with open(model_path + 'lead'+str(lead)+'_20_100.pkl','wb') as f:
        pickle.dump(rf,f)

    y_pred_valid_rf = rf.predict(x_valid)
    #y_pred_valid_rf = np.exp(rf.predict(x_valid)) # Log transformation (uncomment for transformation of output)
    valid_error_rf = np.array(y_valid) - y_pred_valid_rf

    valid_rmse_rf = np.sqrt(np.mean(valid_error_rf**2))
        
    et = time.time()
    elapsed_time = et - st
    
    print("Lead "+str(lead)+" completed!")
    print("Validation RMSE: %s (execution time: %s s)"%(valid_rmse_rf, elapsed_time))
    
    # Test errors of NWP forecasts (before post-processing)
    test_error_forecast = np.array(y_test)-np.array(df_test_lead['vis_ensemble_mean'])
    
    # Test error of post-processed forecasts
    y_pred_test_rf = rf.predict(x_test)
    #y_pred_test_rf = np.exp(rf.predict(x_test)) # Log transformation (uncomment for transformation of output)
    test_error_rf = np.array(y_test)-y_pred_test_rf
    
    # Create a dataframe for storing test results
    df_result_test = df_test_lead[['base', 'lead_hr', 'forecast_time', 'time_in_day', 'vis_obs', 'vis_ensemble_mean']]
    df_result_test['vis_rf'] = y_pred_test_rf
    df_result_test['error_nwp'] = test_error_forecast
    df_result_test['error_rf'] = test_error_rf
    df_result_test.to_csv(result_export_path + 'results_lead'+str(lead)+"_20_100.csv", index=False)
    # File format: results_lead[lead time]_[max_depth]_[n_estimators].csv

Lead 6 completed!
Validation RMSE: 14474.521920631723 (execution time: 30.666905879974365 s)


<ipython-input-15-d6ffd600f425>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['vis_rf'] = y_pred_test_rf
<ipython-input-15-d6ffd600f425>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['error_nwp'] = test_error_forecast
<ipython-input-15-d6ffd600f425>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Lead 12 completed!
Validation RMSE: 14378.935527990134 (execution time: 42.79184627532959 s)


<ipython-input-15-d6ffd600f425>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['vis_rf'] = y_pred_test_rf
<ipython-input-15-d6ffd600f425>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['error_nwp'] = test_error_forecast
<ipython-input-15-d6ffd600f425>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Lead 18 completed!
Validation RMSE: 17838.260861046707 (execution time: 29.83226180076599 s)


...

<ipython-input-15-d6ffd600f425>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['vis_rf'] = y_pred_test_rf
<ipython-input-15-d6ffd600f425>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['error_nwp'] = test_error_forecast
<ipython-input-15-d6ffd600f425>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Lead 114 completed!
Validation RMSE: 17942.453144462983 (execution time: 29.588733911514282 s)


<ipython-input-15-d6ffd600f425>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['vis_rf'] = y_pred_test_rf
<ipython-input-15-d6ffd600f425>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['error_nwp'] = test_error_forecast
<ipython-input-15-d6ffd600f425>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Lead 120 completed!
Validation RMSE: 16259.089793656813 (execution time: 41.87837600708008 s)


<ipython-input-15-d6ffd600f425>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['vis_rf'] = y_pred_test_rf
<ipython-input-15-d6ffd600f425>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result_test['error_nwp'] = test_error_forecast
<ipython-input-15-d6ffd600f425>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-